# HW4 Trees and Forests

Official instructions:

https://www.cs.tufts.edu/cs/135/2024f/Assignments/hw4.html

This is the *starter code* notebook for Problems 1 and 2, looking at sentiment classification using bag-of-words features of product reviews.

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [3]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [4]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree

In [5]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

In [6]:
# TODO fix to path on your local system
DATA_DIR = os.path.join("../data_product_reviews/")

### Load training

In [7]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))

Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [8]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))

Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [9]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))

Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [10]:
vocab_list = x_tr_df.columns.tolist()

In [11]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)

good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [12]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])

In [13]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])

In [14]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [ ]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', 
    max_depth=3, min_samples_split=2, min_samples_leaf=1, 
    random_state=101)

### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [ ]:
# TODO
# simple_tree.fit(FIXME, FIXME)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [ ]:
# TODO 
# call pretty_print_sklearn_tree on simple_tree, providing feature_names=vocab_list

## 1B : Find best Decision Tree with grid search

In [15]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)

In [16]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )

**TODO Build the Grid Search** in the next coding cell

Follow the 1B instructions carefully

In [ ]:
tree_grid_searcher = None # TODO fixme use GridSearchCV

### Do the search!


In [ ]:
start_time_sec = time.time()
# TODO call me: tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [ ]:
tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
print("Grid search of %3d configurations done after %6.1f sec" % (
    tree_search_results_df.shape[0], elapsed_time_sec))

### Display search results

This block will make a pretty printed table of the results of your grid search

In [ ]:
pd.set_option('display.precision', 4)
tree_keys = ['param_max_depth', 'param_min_samples_leaf']
tree_search_results_df.sort_values(tree_keys, inplace=True)
tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]

In [ ]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")

### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [ ]:
best_tree = base_tree # TODO call set_params using the best_params_ found by your searcher
best_tree.fit(x_tr_NF, y_tr_N)

### Interpret the best decision tree

In [ ]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [ ]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

### Fit the forest

**TODO Train on the training set** in the next coding cell

In [ ]:
# TODO FIXME call fit, like: `simple_forest.fit(None, None)`

## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

## 2C: Best Random Forest via grid search

Follow the instructions and using what you learn in 1B to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.)

If yours runs significantly longer, try this out on Google Colab instead.

In [ ]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)


In [ ]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[100],
    random_state=[101],
    )

In [ ]:
# TODO construct a GridSearchCV object like you did above.

forest_searcher = None 

### Do the search!

In [ ]:
#TODO

### Build dataframe of results

In [ ]:
#TODO

### Display search results

In [ ]:
#TODO

### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [ ]:
#TODO

### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|